# Meta Programming

In [115]:
prog = "1 + 1"

"1 + 1"

In [116]:
@doc parse

```
parse(str, start; greedy=true, raise=true)
```

Parse the expression string and return an expression (which could later be passed to eval for execution). `start` is the index of the first character to start parsing. If `greedy` is `true` (default), `parse` will try to consume as much input as it can; otherwise, it will stop as soon as it has parsed a valid expression. Incomplete but otherwise syntactically valid expressions will return `Expr(:incomplete, "(error message)")`. If `raise` is `true` (default), syntax errors other than incomplete expressions will raise an error. If `raise` is `false`, `parse` will return an expression that will raise an error upon evaluation.

```
parse(str; raise=true)
```

Parse the expression string greedily, returning a single expression. An error is thrown if there are additional characters after the first expression. If `raise` is `true` (default), syntax errors will raise an error; otherwise, `parse` will return an expression that will raise an error upon evaluation.

```
parse(type, str, [base])
```

Parse a string as a number. If the type is an integer type, then a base can be specified (the default is 10). If the type is a floating point type, the string is parsed as a decimal floating point number. If the string does not contain a valid number, an error is raised.


In [117]:
ex1 = parse(prog)

:(1 + 1)

In [118]:
typeof(ex1)

Expr

In [119]:
@doc Expr

No documentation found.

**Summary:**

```julia
type Expr <: Any
```

**Fields:**

```julia
head :: Symbol
args :: Array{Any,1}
typ  :: Any
```


In [120]:
(ex1.head, ex1.args, ex1.typ)

(:call,Any[:+,1,1],Any)

In [121]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [122]:
ex1 == ex2

true

The key point here is that Julia code is internally represented as a data structure that is accessible from the language itself.

In [123]:
@doc dump

```
dump(x)
```

Show all user-visible structure of a value.


In [124]:
dump(ex2)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


In [125]:
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [126]:
dump(ex3)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol /
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Int64 4
        3: Int64 4
      typ: Any
    3: Int64 2
  typ: Any


In [127]:
@doc Meta.show_sexpr

No documentation found.

`Base.Meta.show_sexpr` is a generic `Function`.

```julia
# 5 methods for generic function "show_sexpr":
show_sexpr(io::IO, ex::QuoteNode, indent::Int64) at meta.jl:31
show_sexpr(io::IO, ex::Expr, indent::Int64) at meta.jl:37
show_sexpr(io::IO, ex) at meta.jl:25
show_sexpr(io::IO, ex, indent::Int64) at meta.jl:26
show_sexpr(ex) at meta.jl:24
```


In [128]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

In [129]:
:foo == symbol("foo")

true

In [130]:
@doc symbol

```
symbol(x...) -> Symbol
```

Create a `Symbol` by concatenating the string representations of the arguments together.


In [131]:
symbol(:var,'_',"sym")

:var_sym

In [132]:
wow = :(::)

:(::)

In [133]:
dump(wow)

Symbol ::


In [134]:
:(a+b*c+1)

:(a + b * c + 1)

In [135]:
:(a + b*c + 1)  ==
       parse("a + b*c + 1") ==
       Expr(:call, :+, :a, Expr(:call, :*, :b, :c), 1)

true

In [136]:
ex = quote
   x = 1
   y = 2
   x + y
end

quote  # In[136], line 2:
    x = 1 # In[136], line 3:
    y = 2 # In[136], line 4:
    x + y
end

In [137]:
a = 1
ex = :($a + b)

:(1 + b)

In [138]:
ex = :(a in $:((1,2,3)) )

:($(Expr(:in, :a, :((1,2,3)))))

In [139]:
dump(ex)

Expr 
  head: Symbol in
  args: Array(Any,(2,))
    1: Symbol a
    2: Expr 
      head: Symbol tuple
      args: Array(Any,(3,))
        1: Int64 1
        2: Int64 2
        3: Int64 3
      typ: Any
  typ: Any


In [140]:
eval(ex)

true

In [141]:
:(a in $:((1,2,3))) == :(a in $(:((1,2,3))))

true

In [142]:
function make_expr(op, opr1, opr2)
    opr1f, opr2f = map((opr1, opr2)) do x
        isa(x, Number) ? 2*x : x
    end
    
    Expr(:call, op, opr1f, opr2f)
end

make_expr (generic function with 1 method)

In [143]:
make_expr(:+, 2, :x)

:(4 + x)

In [144]:
macro sayhello(name)
   return :( println("Hello, ", $name) )
end

In [145]:
@sayhello "Julia"

Hello, Julia


In [146]:
@doc macroexpand

```
macroexpand(x)
```

Takes the expression `x` and returns an equivalent expression with all macros removed (expanded).


In [147]:
macroexpand( :(@sayhello("human")) )

:(println("Hello, ","human"))

In [148]:
macro showarg(x)
   show(x)
   # ... remainder of macro, returning an expression
end

In [149]:
@showarg 1+2

:(1 + 2)

In [150]:
macroexpand(:(@assert a==b))

:(if a == b
        nothing
    else 
        Base.throw(Base.Main.Base.AssertionError("a == b"))
    end)

In [151]:
module MyModule

macro wrong_time(ex)
  return quote
    local t0 = time()
    local val = $ex
    local t1 = time()
    println("elapsed time: ", t1-t0, " seconds")
    val
  end
end

macro right_time(ex)
  return quote
    local t0 = time()
    local val = $(esc(ex))
    local t1 = time()
    println("elapsed time: ", t1-t0, " seconds")
    val
  end
end
end

MyModule

In [152]:
for op = (:×, :⋅, :∧)
    @eval ($op)(a,b) = @sprintf "(%s %s %s)" a string($op) b
    @eval ($op)(a,b,c) = ($op)(($op)(a,b),c)
end
(2 ⋅ 3, 2 × 3, 2 ∧ 3)

(6,"(2 cross 3)","(2 ∧ 3)")

In [153]:
@which 2 + 2

+(x::Int64, y::Int64) at int.jl:8

In [154]:
@less 1 + 1

# This file is a part of Julia. License is MIT: http://julialang.org/license

## integer arithmetic ##

const IntTypes = (Int8, UInt8, Int16, UInt16, Int32, UInt32,
                  Int64, UInt64, Int128, UInt128)

+(x::Int, y::Int) = box(Int,add_int(unbox(Int,x),unbox(Int,y)))
<(x::Int, y::Int) = slt_int(unbox(Int,x),unbox(Int,y))

for T in IntTypes
    @eval begin
        -(x::$T) = box($T,neg_int(unbox($T,x)))

        if !($T === Int)  # don't overwrite definition from line 8
            +(x::$T, y::$T) = box($T, add_int(unbox($T,x),unbox($T,y)))
        end
        -(x::$T, y::$T) = box($T, sub_int(unbox($T,x),unbox($T,y)))
        *(x::$T, y::$T) = box($T, mul_int(unbox($T,x),unbox($T,y)))
    end
end

/(x::Integer, y::Integer) = float(x)/float(y)
inv(x::Integer) = float(one(x))/float(x)

isodd(n::Integer) = rem(n,2) != 0
iseven(n::Integer) = !isodd(n)

signbit(x::Integer) = x < 0
signbit(x::Unsigned) = false

for T in (Int8,Int16,Int32,Int64,Int128)
    @eval flipsign(x::$T, y:

In [155]:
@edit 1 + 1

Unknown editor: no line number information passed.
The method is defined at line 8.


In [156]:
clipboard(:(1+1))
# Paste then we get
1 + 1

2

In [157]:
apropos("IPv6")

parseip
IPv6
listen
recvfrom


In [158]:
methods(+)

# 171 methods for generic function "+":
+(x::Bool) at bool.jl:33
+(x::Bool, y::Bool) at bool.jl:36
+(y::AbstractFloat, x::Bool) at bool.jl:46
+(x::Int64, y::Int64) at int.jl:8
+(x::Int8, y::Int8) at int.jl:16
+(x::UInt8, y::UInt8) at int.jl:16
+(x::Int16, y::Int16) at int.jl:16
+(x::UInt16, y::UInt16) at int.jl:16
+(x::Int32, y::Int32) at int.jl:16
+(x::UInt32, y::UInt32) at int.jl:16
+(x::UInt64, y::UInt64) at int.jl:16
+(x::Int128, y::Int128) at int.jl:16
+(x::UInt128, y::UInt128) at int.jl:16
+(x::Integer, y::Ptr{T}) at pointer.jl:77
+(x::Float32, y::Float32) at float.jl:207
+(x::Float64, y::Float64) at float.jl:208
+(z::Complex{T<:Real}, w::Complex{T<:Real}) at complex.jl:111
+(x::Bool, z::Complex{Bool}) at complex.jl:118
+(z::Complex{Bool}, x::Bool) at complex.jl:119
+(x::Bool, z::Complex{T<:Real}) at complex.jl:125
+(z::Complex{T<:Real}, x::Bool) at complex.jl:126
+(x::Real, z::Complex{Bool}) at complex.jl:132
+(z::Complex{Bool}, x::Real) at complex.jl:133
+(x::Real, z::Complex{T<:Real}) at complex.jl:144
+(z::Complex{T<:Real}, x::Real) at complex.jl:145
+(x::Rational{T<:Integer}, y::Rational{T<:Integer}) at rational.jl:179
+(x::Bool, A::AbstractArray{Bool,N}) at arraymath.jl:136
+(x::Integer, y::Char) at char.jl:43
+(a::Float16, b::Float16) at float16.jl:136
+(x::BigInt, y::BigInt) at gmp.jl:256
+(a::BigInt, b::BigInt, c::BigInt) at gmp.jl:279
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) at gmp.jl:285
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) at gmp.jl:292
+(x::BigInt, c::Union{UInt16,UInt32,UInt64,UInt8}) at gmp.jl:304
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigInt) at gmp.jl:308
+(x::BigInt, c::Union{Int16,Int32,Int64,Int8}) at gmp.jl:320
+(c::Union{Int16,Int32,Int64,Int8}, x::BigInt) at gmp.jl:321
+(x::BigFloat, y::BigFloat) at mpfr.jl:208
+(x::BigFloat, c::Union{UInt16,UInt32,UInt64,UInt8}) at mpfr.jl:215
+(c::Union{UInt16,UInt32,UInt64,UInt8}, x::BigFloat) at mpfr.jl:219
+(x::BigFloat, c::Union{Int16,Int32,Int64,Int8}) at mpfr.jl:223
+(c::Union{Int16,Int32,Int64,Int8}, x::BigFloat) at mpfr.jl:227
+(x::BigFloat, c::Union{Float16,Float32,Float64}) at mpfr.jl:231
+(c::Union{Float16,Float32,Float64}, x::BigFloat) at mpfr.jl:235
+(x::BigFloat, c::BigInt) at mpfr.jl:239
+(c::BigInt, x::BigFloat) at mpfr.jl:243
+(a::BigFloat, b::BigFloat, c::BigFloat) at mpfr.jl:379
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) at mpfr.jl:385
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) at mpfr.jl:392
+(x::Irrational{sym}, y::Irrational{sym}) at irrationals.jl:72
+(x::Number) at operators.jl:73
+{T<:Number}(x::T<:Number, y::T<:Number) at promotion.jl:211
+{T<:AbstractFloat}(x::Bool, y::T<:AbstractFloat) at bool.jl:43
+(x::Number, y::Number) at promotion.jl:167
+(r1::OrdinalRange{T,S}, r2::OrdinalRange{T,S}) at operators.jl:330
+{T<:AbstractFloat}(r1::FloatRange{T<:AbstractFloat}, r2::FloatRange{T<:AbstractFloat}) at operators.jl:337
+{T<:AbstractFloat}(r1::LinSpace{T<:AbstractFloat}, r2::LinSpace{T<:AbstractFloat}) at operators.jl:356
+(r1::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}, r2::Union{FloatRange{T<:AbstractFloat},LinSpace{T<:AbstractFloat},OrdinalRange{T,S}}) at operators.jl:369
+(x::Ptr{T}, y::Integer) at pointer.jl:75
+{S,T}(A::Range{S}, B::Range{T}) at arraymath.jl:69
+{S,T}(A::Range{S}, B::AbstractArray{T,N}) at arraymath.jl:87
+(A::BitArray{N}, B::BitArray{N}) at bitarray.jl:834
+{T}(B::BitArray{2}, J::UniformScaling{T}) at linalg/uniformscaling.jl:28
+(A::Array{T,2}, B::Diagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Bidiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::Tridiagonal{T}) at linalg/special.jl:122
+(A::Array{T,2}, B::SymTridiagonal{T}) at linalg/special.jl:131
+(A::Array{T,2}, B::Base.LinAlg.AbstractTriangular{T,S<:AbstractArray{T,2}}) at linalg/special.jl:159
+(A::Array{T,N}, B::SparseMatrixCSC{Tv,Ti<:Integer}) at sparse/sparsematrix.jl:1019
+{P<:Union{Base.Dates.CompoundPeriod,Base.Dates.Period}}(x::Union{De

In [159]:
methodswith(Real)

326-element Array{Method,1}:
 rem{T<:Real}(x::T<:Real, y::T<:Real) at promotion.jl:232                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
 rem(x::Real, y::Real) at promotion.jl:187                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
 rem{P<:Base.Dates.Period}(x::P<:Base.Dates.Period, y::Real) at dates/periods.jl:49                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
 *(x::Real, z::Complex{Bool}) at complex.jl:140                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 *(z::Complex{Bool}, x::Real) at complex.jl:141                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
 *(x::Real, z::Complex{T<:Real}) at complex.jl:152                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [160]:
methodswith(Real, +)

4-element Array{Method,1}:
 +(x::Real, z::Complex{Bool}) at complex.jl:132   
 +(z::Complex{Bool}, x::Real) at complex.jl:133   
 +(x::Real, z::Complex{T<:Real}) at complex.jl:144
 +(z::Complex{T<:Real}, x::Real) at complex.jl:145

In [161]:
object_id(+)

0x489bea67662baecd

In [162]:
@which getfield

Core

In [181]:
@doc getfield

```
getfield(value, name::Symbol)
```

Extract a named field from a `value` of composite type. The syntax `a.b` calls `getfield(a, :b)`, and the syntax `a.(b)` calls `getfield(a, b)`.


In [163]:
type Experiment
    a::Int64
    b::Function
    c::AbstractString
end

In [164]:
expm = Experiment(100, +, "A String")

Experiment(100,+,"A String")

In [165]:
expm.(:c)

"A String"

In [166]:
expm.c

"A String"

In [167]:
symbolc = "c"
expm.(symbol(symbolc))

"A String"

In [168]:
fieldoffsets(Experiment)

3-element Array{Int64,1}:
  0
  8
 16

In [169]:
structinfo(T) = [zip(fieldoffsets(T),fieldnames(T),T.types)...]

structinfo (generic function with 1 method)

In [170]:
structinfo(Experiment)

3-element Array{Tuple{Int64,Symbol,DataType},1}:
 (0,:a,Int64)          
 (8,:b,Function)       
 (16,:c,AbstractString)

In [171]:
expm.b(1, 3)

4

In [172]:
method_exists(+, Tuple{Real})

true

In [173]:
applicable(+, 1)

true

In [174]:
applicable(+, "1")

false

In [175]:
@doc invoke

```
invoke(f, (types...), args...)
```

Invoke a method for the given generic function matching the specified types (as a tuple), on the specified arguments. The arguments must be compatible with the specified types. This allows invoking a method other than the most specific matching method, which is useful when the behavior of a more general definition is explicitly needed (often as part of the implementation of a more specific method of the same function).


In [176]:
invoke(+, (Float64, Float64), 1.0, 1.0)

2.0

In [177]:
[1:5;] |> x->x.^2 |> sum |> inv == inv(sum([1:5;].^2))

true

In [178]:
@doc call

```
call(x, args...)
```

If `x` is not a `Function`, then `x(args...)` is equivalent to `call(x, args...)`. This means that function-like behavior can be added to any type by defining new `call` methods.


In [179]:
call(m::Experiment, s::AbstractString) = parse(s)

call (generic function with 1087 methods)

In [180]:
expm("123")

123

In [184]:
@doc @gensym

```
@gensym
```

Generates a gensym symbol for a variable. For example, `@gensym x y` is transformed into `x = gensym("x"); y = gensym("y")`.


In [195]:
@doc @elapsed

```
@elapsed
```

A macro to evaluate an expression, discarding the resulting value, instead returning the number of seconds it took to execute as a floating-point number.


In [186]:
@elapsed sleep(1)

1.046363498

In [196]:
@doc @allocated

```
@allocated
```

A macro to evaluate an expression, discarding the resulting value, instead returning the total number of bytes allocated during evaluation of the expression. Note: the expression is evaluated inside a local function, instead of the current context, in order to eliminate the effects of compilation, however, there still may be some allocations due to JIT compilation. This also makes the results inconsistent with the `@time` macros, which do not try to adjust for the effects of compilation.


In [187]:
@allocated [1, 2, 3]

96

In [188]:
@allocated [1]

80

In [189]:
@allocated 1

0

In [194]:
(@allocated(collect(range(0, 1000))) - @allocated([]))/1000

8.048

In [200]:
(ff, fl) = functionloc(+, (Real, Real))

("/opt/homebrew-cask/Caskroom/julia/0.4.5/Julia-0.4.5.app/Contents/Resources/julia/bin/../share/julia/base/promotion.jl",211)

In [201]:
less(ff, fl)

# This file is a part of Julia. License is MIT: http://julialang.org/license

## type join (closest common ancestor, or least upper bound) ##

typejoin() = Bottom
typejoin(t::ANY) = t
typejoin(t::ANY, ts...) = typejoin(t, typejoin(ts...))
function typejoin(a::ANY, b::ANY)
    if isa(a,TypeConstructor); a = a.body; end
    if isa(b,TypeConstructor); b = b.body; end
    if a <: b
        return b
    elseif b <: a
        return a
    end
    if isa(a,TypeVar)
        return typejoin(a.ub, b)
    end
    if isa(b,TypeVar)
        return typejoin(a, b.ub)
    end
    if isa(a,Union) || isa(b,Union)
        u = Union{a, b}
        if !isa(u,Union)
            return u
        end
        return reduce(typejoin, Bottom, u.types)
    end
    if a <: Tuple
        if !(b <: Tuple)
            return Any
        end
        ap, bp = a.parameters, b.parameters
        la = length(ap)::Int; lb = length(bp)::Int
        if la==0 || lb==0
            return Tuple
        end
        if la < lb
   

In [202]:
less(+, (Real, Real))

# This file is a part of Julia. License is MIT: http://julialang.org/license

## type join (closest common ancestor, or least upper bound) ##

typejoin() = Bottom
typejoin(t::ANY) = t
typejoin(t::ANY, ts...) = typejoin(t, typejoin(ts...))
function typejoin(a::ANY, b::ANY)
    if isa(a,TypeConstructor); a = a.body; end
    if isa(b,TypeConstructor); b = b.body; end
    if a <: b
        return b
    elseif b <: a
        return a
    end
    if isa(a,TypeVar)
        return typejoin(a.ub, b)
    end
    if isa(b,TypeVar)
        return typejoin(a, b.ub)
    end
    if isa(a,Union) || isa(b,Union)
        u = Union{a, b}
        if !isa(u,Union)
            return u
        end
        return reduce(typejoin, Bottom, u.types)
    end
    if a <: Tuple
        if !(b <: Tuple)
            return Any
        end
        ap, bp = a.parameters, b.parameters
        la = length(ap)::Int; lb = length(bp)::Int
        if la==0 || lb==0
            return Tuple
        end
        if la < lb
   

In [204]:
methods(Experiment)

4-element Array{Any,1}:
 call(::Type{Experiment}, a::Int64, b::Function, c::AbstractString) at In[163]:2
 call(::Type{Experiment}, a, b, c) at In[163]:2                                 
 call{T}(::Type{T}, arg) at essentials.jl:56                                    
 call{T}(::Type{T}, args...) at essentials.jl:57                                

In [205]:
less(Experiment, (Int64, Function, AbstractString))

LoadError: LoadError: could not find source file for function
while loading In[205], in expression starting on line 1

In [208]:
expand(:(x + 3y))

:(x + 3y)

In [217]:
function af(x)
    x + 1
end
expand(:(af(3)))

:(af(3))

In [226]:
@code_lowered join(["a", "b"], "#")

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:(args::Any...)], Any[Any[Any[:args,:Any,0]],Any[],0,Any[]], :(begin  # strings/io.jl, line 104:
        return (top(_apply))((top(getfield))(Base,:call),Base.sprint,(top(tuple))(Base.print_joined),args)
    end))))

In [227]:
@code_lowered 1 + 1

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x,:y], Any[Any[Any[:x,:Any,0],Any[:y,:Any,0]],Any[],0,Any[]], :(begin  # int.jl, line 8:
        return (Base.box)(Base.Int,(Base.add_int)((Base.unbox)(Base.Int,x),(Base.unbox)(Base.Int,y)))
    end))))

In [228]:
@code_lowered af(3)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x], Any[Any[Any[:x,:Any,0]],Any[],0,Any[]], :(begin  # In[217], line 2:
        return x + 1
    end))))

In [231]:
@code_typed join(["a", "b"], "#")

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:(args::Any...)], Any[Any[Any[:args,Tuple{Array{ASCIIString,1},ASCIIString},0],Any[symbol("##args#8826"),Tuple{Array{ASCIIString,1},ASCIIString},0]],Any[],Any[],Any[]], :(begin  # strings/io.jl, line 104:
        return (Base.sprint)(0,Base.print_joined,(top(getfield))(args::Tuple{Array{ASCIIString,1},ASCIIString},1)::Array{ASCIIString,1},(top(getfield))(args::Tuple{Array{ASCIIString,1},ASCIIString},2)::ASCIIString)::Union{ASCIIString,UTF8String}
    end::Union{ASCIIString,UTF8String}))))

In [232]:
@code_typed 1 + 1

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x,:y], Any[Any[Any[:x,Int64,0],Any[:y,Int64,0]],Any[],Any[],Any[]], :(begin  # int.jl, line 8:
        return (Base.box)(Base.Int,(Base.add_int)(x::Int64,y::Int64))
    end::Int64))))

In [233]:
@code_typed af(3)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:x], Any[Any[Any[:x,Int64,0]],Any[],Any[],Any[]], :(begin  # In[217], line 2:
        return (Base.box)(Base.Int,(Base.add_int)(x::Int64,1))
    end::Int64))))

In [234]:
@code_warntype join(["a", "b"], "#")

Variables:
  args::Tuple{Array{ASCIIString,1},ASCIIString}
  ##args#8826::Tuple{Array{ASCIIString,1},ASCIIString}

Body:
  begin  # strings/io.jl, line 104:
      return (Base.sprint)(0,Base.print_joined,(top(getfield))(args::Tuple{Array{ASCIIString,1},ASCIIString},1)::Array{ASCIIString,1},(top(getfield))(args::Tuple{Array{ASCIIString,1},ASCIIString},2)::ASCIIString)::UNION{ASCIISTRING,UTF8STRING}
  end::UNION{ASCIISTRING,UTF8STRING}


In [235]:
@code_warntype 1 + 1

Variables:
  x::Int64
  y::Int64

Body:
  begin  # int.jl, line 8:
      return (Base.box)(Base.Int,(Base.add_int)(x::Int64,y::Int64))
  end::Int64


In [236]:
@code_warntype af(3)

Variables:
  x::Int64

Body:
  begin  # In[217], line 2:
      return (Base.box)(Base.Int,(Base.add_int)(x::Int64,1))
  end::Int64


In [239]:
@code_warntype (1 + 2 * af(3))

Variables:
  x::Int64
  y::Int64

Body:
  begin  # int.jl, line 8:
      return (Base.box)(Base.Int,(Base.add_int)(x::Int64,y::Int64))
  end::Int64


In [240]:
@code_llvm join(["a", "b"], "#")


define %jl_value_t* @julia_join_23846(%jl_value_t*, %jl_value_t**, i32) {
ifcont:
  %3 = alloca [7 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [7 x %jl_value_t*]* %3, i64 0, i64 0
  %4 = getelementptr [7 x %jl_value_t*]* %3, i64 0, i64 2
  %5 = getelementptr [7 x %jl_value_t*]* %3, i64 0, i64 3
  store %jl_value_t* inttoptr (i64 10 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %6 = load %jl_value_t*** @jl_pgcstack, align 8
  %7 = getelementptr [7 x %jl_value_t*]* %3, i64 0, i64 1
  %.c = bitcast %jl_value_t** %6 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %7, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %4, align 8
  store %jl_value_t* null, %jl_value_t** %5, align 8
  %8 = getelementptr [7 x %jl_value_t*]* %3, i64 0, i64 4
  store %jl_value_t* null, %jl_value_t** %8, align 8
  %9 = getelementptr [7 x %jl_value_t*]* %3, i64 0, i64 5
  store %jl_value_t* null, %jl_value_t** %9, align 8
  

In [241]:
@code_llvm 1 + 1


define i64 @"julia_+_23847"(i64, i64) {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


In [242]:
@code_llvm af(3)


define i64 @julia_af_23848(i64) {
top:
  %1 = add i64 %0, 1
  ret i64 %1
}


In [243]:
@code_native join(["a", "b"], "#")

	.section	__TEXT,__text,regular,pure_instructions
Filename: strings/io.jl
Source line: 104
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 104
	pushq	%r14
	pushq	%rbx
	subq	$64, %rsp
	movq	$10, -72(%rbp)
	movabsq	$jl_pgcstack, %r14
	movq	(%r14), %rax
	movq	%rax, -64(%rbp)
	leaq	-72(%rbp), %rax
	movq	%rax, (%r14)
	xorps	%xmm0, %xmm0
	movups	%xmm0, -56(%rbp)
	movups	%xmm0, -40(%rbp)
	movq	$0, -24(%rbp)
	movabsq	$jl_f_tuple, %rax
	xorl	%edi, %edi
	callq	*%rax
Source line: 104
	movabsq	$jl_gc_alloc_1w, %rcx
	movabsq	$4486815064, %rdx       ## imm = 0x10B6F5D58
	movabsq	$4503813048, %rsi       ## imm = 0x10C72BBB8
	movabsq	$4505050840, %rdi       ## imm = 0x10C859ED8
Source line: 104
	movq	%rax, -56(%rbp)
Source line: 104
	movq	(%rdi), %rax
	movq	%rax, -48(%rbp)
	movq	(%rsi), %rax
	movq	%rax, -40(%rbp)
	movq	(%rdx), %rbx
	callq	*%rcx
Source line: 104
	leaq	-48(%rbp), %rsi
Source line: 104
	movabsq	$jl_f_apply, %rcx
	movabsq	$4471812976, %rdx       ## imm = 0x10A8A7370
	movq	%rdx, -8(%rax)
	movq	%

In [244]:
@code_native 1 + 1

	.section	__TEXT,__text,regular,pure_instructions
Filename: int.jl
Source line: 8
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 8
	addq	%rsi, %rdi
	movq	%rdi, %rax
	popq	%rbp
	ret


In [245]:
@code_native af(3)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[217]
Source line: 2
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 2
	leaq	1(%rdi), %rax
	popq	%rbp
	ret


https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming

In [247]:
P = quote
     a = 2
     b = 3
     c = 4
     d = 5
     e = sum([a,b,c,d])
end

quote  # In[247], line 2:
    a = 2 # In[247], line 3:
    b = 3 # In[247], line 4:
    c = 4 # In[247], line 5:
    d = 5 # In[247], line 6:
    e = sum([a,b,c,d])
end

In [248]:
for (n, expr) in enumerate(P.args)
  println(n, ": ", expr)
end

1:  # In[247], line 2:
2: a = 2
3:  # In[247], line 3:
4: b = 3
5:  # In[247], line 4:
6: c = 4
7:  # In[247], line 5:
8: d = 5
9:  # In[247], line 6:
10: e = sum([a,b,c,d])


In [249]:
quote s = $(sin(1) + cos(1))
end

quote  # In[249], line 1:
    s = 1.3817732906760363
end

In [250]:
macro p(n)
    if typeof(n) == Expr 
       println(n.args)
    end
    eval(n)
end

In [251]:
@p 3

3

In [252]:
@p 3 + 4 - 5 * 6 / 7 % 8

Any[:-,:(3 + 4),:(((5 * 6) / 7) % 8)]


2.7142857142857144

In [253]:
macro f(x)
    quote
       s = 4
       (s, $(esc(s)))
    end
end

In [254]:
s = 0
@f s

(4,0)

In [255]:
@doc esc

```rst
..  esc(e::ANY)

Only valid in the context of an ``Expr`` returned from a macro. Prevents the macro hygiene pass from turning embedded variables into gensym variables. See the :ref:`man-macros`
section of the Metaprogramming chapter of the manual for more details and examples.
```


In [263]:
macro dotimes(n, body)
    quote
        for i = 1:$(esc(n))
            @printf "%d\t" i
            $(esc(body))
        end
    end
end
@dotimes 3 println("......")

1	......
2	......
3	......


In [264]:
macro until(condition, block)
    quote
        while true
            $(esc(block))
            if $(esc(condition))
                break
            end
        end
    end
end

In [269]:
i = 0
@printf "|"
@until i == 10 begin
    i += 1
    @printf "%d|" i
end

|1|2|3|4|5|6|7|8|9|10|

In [278]:
apropos("⋅") # \cdot

At_mul_Bt
Ac_mul_B
At_mul_B
A_mul_B!
A_mul_Bt
A_mul_Bc
dot


In [290]:
@doc dot

```
dot(x, y)
⋅(x,y)
```

Compute the dot product. For complex vectors, the first vector is conjugated.


In [279]:
apropos("×") # \times

cross


In [291]:
@doc cross

```
cross(x, y)
×(x,y)
```

Compute the cross product of two 3-vectors.


In [292]:
@which cross

Base.LinAlg

In [293]:
apropos("∧") # \wedge

In [294]:
"∧" == "^"

false

In [295]:
1^2

1

In [296]:
1∧2

"(1 ∧ 2)"

In [297]:
typeof(1∧2)

UTF8String

In [298]:
1∫2

LoadError: LoadError: UndefVarError: ∫2 not defined
while loading In[298], in expression starting on line 1

In [299]:
∧2

LoadError: LoadError: syntax: "∧" is not a unary operator
while loading In[299], in expression starting on line 1

In [300]:
1∧

LoadError: LoadError: syntax: incomplete: premature end of input
while loading In[300], in expression starting on line 1

In [302]:
@which ∧

Main

In [306]:
@doc Main.∧

No documentation found.

`∧` is a generic `Function`.

```julia
# 2 methods for generic function "∧":
∧(a, b) at In[152]:2
∧(a, b, c) at In[152]:3
```


In [310]:
@doc Base.Operators

No documentation found.

`Base.Operators` is of type `Module`:

**Summary:**

```julia
type Module <: Any
```

**Fields:**

```julia
name   :: Symbol
parent :: Any
```


In [313]:
Base.Operators.⋅

dot (generic function with 7 methods)

See [operator precedence table](https://github.com/JuliaLang/julia/blob/master/src/julia-parser.scm) in Julia source code.

```scheme
;; Operator precedence table, lowest at top

;; note: there are some strange-looking things in here because
;; the way the lexer works, every prefix of an operator must also
;; be an operator.
(define prec-assignment
  '(= := += -= *= /= //= .//= .*= ./= |\\=| |.\\=| ^= .^= ÷= .÷= %= .%= |\|=| &= $= => <<= >>= >>>= ~ |.+=| |.-=|))
(define prec-conditional '(?))
(define prec-arrow       '(-- --> ← → ↔ ↚ ↛ ↠ ↣ ↦ ↮ ⇎ ⇏ ⇒ ⇔ ⇴ ⇶ ⇷ ⇸ ⇹ ⇺ ⇻ ⇼ ⇽ ⇾ ⇿ ⟵ ⟶ ⟷ ⟷ ⟹ ⟺ ⟻ ⟼ ⟽ ⟾ ⟿ ⤀ ⤁ ⤂ ⤃ ⤄ ⤅ ⤆ ⤇ ⤌ ⤍ ⤎ ⤏ ⤐ ⤑ ⤔ ⤕ ⤖ ⤗ ⤘ ⤝ ⤞ ⤟ ⤠ ⥄ ⥅ ⥆ ⥇ ⥈ ⥊ ⥋ ⥎ ⥐ ⥒ ⥓ ⥖ ⥗ ⥚ ⥛ ⥞ ⥟ ⥢ ⥤ ⥦ ⥧ ⥨ ⥩ ⥪ ⥫ ⥬ ⥭ ⥰ ⧴ ⬱ ⬰ ⬲ ⬳ ⬴ ⬵ ⬶ ⬷ ⬸ ⬹ ⬺ ⬻ ⬼ ⬽ ⬾ ⬿ ⭀ ⭁ ⭂ ⭃ ⭄ ⭇ ⭈ ⭉ ⭊ ⭋ ⭌ ￩ ￫))
(define prec-lazy-or     '(|\|\||))
(define prec-lazy-and    '(&&))
(define prec-comparison
  '(> < >= ≥ <= ≤ == === ≡ != ≠ !== ≢ |.>| |.<| |.>=| |.≥| |.<=| |.≤| |.==| |.!=| |.≠| |.=| |.!| |<:| |>:| ∈ ∉ ∋ ∌ ⊆ ⊈ ⊂ ⊄ ⊊ ∝ ∊ ∍ ∥ ∦ ∷ ∺ ∻ ∽ ∾ ≁ ≃ ≄ ≅ ≆ ≇ ≈ ≉ ≊ ≋ ≌ ≍ ≎ ≐ ≑ ≒ ≓ ≔ ≕ ≖ ≗ ≘ ≙ ≚ ≛ ≜ ≝ ≞ ≟ ≣ ≦ ≧ ≨ ≩ ≪ ≫ ≬ ≭ ≮ ≯ ≰ ≱ ≲ ≳ ≴ ≵ ≶ ≷ ≸ ≹ ≺ ≻ ≼ ≽ ≾ ≿ ⊀ ⊁ ⊃ ⊅ ⊇ ⊉ ⊋ ⊏ ⊐ ⊑ ⊒ ⊜ ⊩ ⊬ ⊮ ⊰ ⊱ ⊲ ⊳ ⊴ ⊵ ⊶ ⊷ ⋍ ⋐ ⋑ ⋕ ⋖ ⋗ ⋘ ⋙ ⋚ ⋛ ⋜ ⋝ ⋞ ⋟ ⋠ ⋡ ⋢ ⋣ ⋤ ⋥ ⋦ ⋧ ⋨ ⋩ ⋪ ⋫ ⋬ ⋭ ⋲ ⋳ ⋴ ⋵ ⋶ ⋷ ⋸ ⋹ ⋺ ⋻ ⋼ ⋽ ⋾ ⋿ ⟈ ⟉ ⟒ ⦷ ⧀ ⧁ ⧡ ⧣ ⧤ ⧥ ⩦ ⩧ ⩪ ⩫ ⩬ ⩭ ⩮ ⩯ ⩰ ⩱ ⩲ ⩳ ⩴ ⩵ ⩶ ⩷ ⩸ ⩹ ⩺ ⩻ ⩼ ⩽ ⩾ ⩿ ⪀ ⪁ ⪂ ⪃ ⪄ ⪅ ⪆ ⪇ ⪈ ⪉ ⪊ ⪋ ⪌ ⪍ ⪎ ⪏ ⪐ ⪑ ⪒ ⪓ ⪔ ⪕ ⪖ ⪗ ⪘ ⪙ ⪚ ⪛ ⪜ ⪝ ⪞ ⪟ ⪠ ⪡ ⪢ ⪣ ⪤ ⪥ ⪦ ⪧ ⪨ ⪩ ⪪ ⪫ ⪬ ⪭ ⪮ ⪯ ⪰ ⪱ ⪲ ⪳ ⪴ ⪵ ⪶ ⪷ ⪸ ⪹ ⪺ ⪻ ⪼ ⪽ ⪾ ⪿ ⫀ ⫁ ⫂ ⫃ ⫄ ⫅ ⫆ ⫇ ⫈ ⫉ ⫊ ⫋ ⫌ ⫍ ⫎ ⫏ ⫐ ⫑ ⫒ ⫓ ⫔ ⫕ ⫖ ⫗ ⫘ ⫙ ⫷ ⫸ ⫹ ⫺ ⊢ ⊣)) ;; plus `in`
(define prec-pipe        '(|\|>| |<\||))
(define prec-colon       '(: |..|))
(define prec-plus        '(+ - ⊕ ⊖ ⊞ ⊟ |.+| |.-| |++| |\|| ∪ ∨ $ ⊔ ± ∓ ∔ ∸ ≂ ≏ ⊎ ⊻ ⊽ ⋎ ⋓ ⧺ ⧻ ⨈ ⨢ ⨣ ⨤ ⨥ ⨦ ⨧ ⨨ ⨩ ⨪ ⨫ ⨬ ⨭ ⨮ ⨹ ⨺ ⩁ ⩂ ⩅ ⩊ ⩌ ⩏ ⩐ ⩒ ⩔ ⩖ ⩗ ⩛ ⩝ ⩡ ⩢ ⩣))
(define prec-bitshift    '(<< >> >>> |.<<| |.>>| |.>>>|))
(define prec-times       '(* / |./| ÷ |.÷| % ⋅ ∘ × |.%| |.*| |\\| |.\\| & ∩ ∧ ⊗ ⊘ ⊙ ⊚ ⊛ ⊠ ⊡ ⊓ ∗ ∙ ∤ ⅋ ≀ ⊼ ⋄ ⋆ ⋇ ⋉ ⋊ ⋋ ⋌ ⋏ ⋒ ⟑ ⦸ ⦼ ⦾ ⦿ ⧶ ⧷ ⨇ ⨰ ⨱ ⨲ ⨳ ⨴ ⨵ ⨶ ⨷ ⨸ ⨻ ⨼ ⨽ ⩀ ⩃ ⩄ ⩋ ⩍ ⩎ ⩑ ⩓ ⩕ ⩘ ⩚ ⩜ ⩞ ⩟ ⩠ ⫛ ⊍ ▷ ⨝ ⟕ ⟖ ⟗))
(define prec-rational    '(// .//))
(define prec-power       '(^ |.^| ↑ ↓ ⇵ ⟰ ⟱ ⤈ ⤉ ⤊ ⤋ ⤒ ⤓ ⥉ ⥌ ⥍ ⥏ ⥑ ⥔ ⥕ ⥘ ⥙ ⥜ ⥝ ⥠ ⥡ ⥣ ⥥ ⥮ ⥯ ￪ ￬))
(define prec-decl        '(|::|))
(define prec-dot         '(|.|))
```